In [18]:
import numpy as np
from scipy.optimize import minimize

def distance(P1, P2):
    """ Compute Euclidean distance between two points """
    return np.linalg.norm(P1 - P2)

def objective(a1):
    """ Objective function: minimize deceleration a1 """
    return a1  # Minimize braking force

def detect_collision(P1, P2, V1, V2, r1, r2):
    """ Detects if a collision will occur with the given object and return deceleration to avoid it."""
    delta_p = P2 - P1
    delta_v = V1 - V2

    # Normalize the vectors
    norm_p = np.linalg.norm(delta_p)
    norm_v = np.linalg.norm(delta_v)

    tol = 1e-6
    # Avoid division by zero
    if norm_p < tol or norm_v < tol:
        return False

    unit_p = delta_p / norm_p  # Normalized ΔP
    unit_v = delta_v / norm_v  # Normalized ΔV

    # Check if unit vectors are nearly identical
    return np.allclose(unit_p, unit_v, atol=tol)

def constraint_finite_difference(a1x, v1, v2, P1, P2, V1, V2, r1, r2):
    """ Ensure Object 1 does not collide with Object 2 while braking """

    print('a1x', a1x)
    if a1x <= 0:
        return -1  # Ensure positive deceleration
    print('v1', v1)
    print('P1', distance(P1, P2))
    t_c = (distance(P1, P2) - (r1 + r2)) / (v1 - v2)
    print('t_c', t_c)
    P1_new = P1 + V1 * t_c - 0.5 * np.array([a1x[0], 0]) * t_c**2
    P2_new = P2 + V2 * t_c  # Object 2 continues normally
    print('P1_new', P1_new)
    print('P2_new', P2_new)
    print('distance', distance(P1_new, P2_new))
    # Ensure new positions do not result in a collision
    return distance(P1_new, P2_new) - (r1 + r2)


# Example input
P1 = np.array([0, 0])
V1 = np.array([60, 60])  # Initial velocity of Object 1
P2 = np.array([5, 5])
V2 = np.array([1, 1])  # Velocity of Object 2
v1 = np.linalg.norm(V1)  # Compute initial speed
v2 = np.linalg.norm(V2)
print('v1', v1)
r1 = 0.5  # Radius of Object 1
r2 = 0.5  # Radius of Object 2

# Initial guess: a1
a1x0 = 0.1  # Start with a small deceleration

# Bounds: (a1 in (0, 1] to ensure positive braking)
bounds = [(1e-3, 1)]  # Prevent zero deceleration

# Solve optimization problem
res = minimize(
    objective, 
    a1x0, 
    bounds=bounds,
    constraints={'type': 'ineq', 'fun': constraint_finite_difference, 'args': (v1, v2, P1, P2, V1, V2, r1, r2)}
)

# Optimal deceleration
optimal_a1 = res.x[0]
print("Optimal Deceleration (Braking):", optimal_a1)

# # Compute braking duration based on optimal deceleration
# optimal_t_d = (s1 - (distance(P1, P2) / (distance(P1, P2) / (distance(P1, P2) / s1 + d_safe / s1)))) / optimal_a1
# print("Braking Duration (t_d):", optimal_t_d)

# # Compute new velocity after braking (only reducing speed, not direction)
# scaling_factor = 1 - (optimal_a1 * optimal_t_d / s1)
# V1_new = scaling_factor * V1
# print("New Velocity After Braking:", V1_new)


v1 84.8528137423857
a1x [0.1]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36538941 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.0001871939422702
a1x [0.1]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36538941 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.0001871939422702
a1x [0.1]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36538941 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.0001871939422702
a1x [0.10000001]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36538941 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.0001871939701665
a1x [0.001]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36565147 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.0000018717660313
a1x [0.001]
v1 84.8528137423857
P1 7.0710678118654755
t_c 0.07276090201378733
P1_new [4.36565147 4.36565412]
P2_new [5.0727609 5.0727609]
distance 1.00

In [8]:
import math
print(math.sqrt(50))

7.0710678118654755


In [11]:

print((math.sqrt(50) - 1) / 2.8284271247461903)

2.146446609406726


In [24]:
import numpy as np
from scipy.optimize import minimize

def distance(P1, P2):
    """ Compute Euclidean distance between two points """
    return np.linalg.norm(P1 - P2)

def detect_collision(P1, P2, V1, V2, r1, r2):
    """
    Checks if a collision is possible based on initial relative motion.
    """
    delta_p = P2 - P1  # Displacement vector
    delta_v = V2 - V1  # Relative velocity

    # Check if objects are moving toward each other
    return np.dot(delta_p, delta_v) < 0

def constraint_no_collision(a1x, P1, P2, V1, V2, r1, r2, t_final):
    """
    Ensures that Object 1 does not collide with Object 2 during the movement.
    """

    # Simulate motion for multiple time steps
    time_steps = np.linspace(0, t_final, num=100)
    print('a1x', a1x)
    for t in time_steps:
        # Update Object 1's position (only decelerating in x-direction)
        new_x = P1[0] + V1[0] * t - 0.5 * a1x[0] * t**2  # Apply braking in x direction
        new_y = P1[1] + V1[1] * t  # Y-direction remains unchanged
        P1_new = np.array([new_x, new_y])

        # Update Object 2's position (continues moving)
        P2_new = P2 + V2 * t

        # Check if collision occurs
        if distance(P1_new, P2_new) < (r1 + r2):
            return distance(P1_new, P2_new) - (r1 + r2)  # Violation of collision constraint

    return distance(P1_new, P2_new) - (r1 + r2)  # No collision

def objective(a1x):
    """ Objective function: minimize deceleration a1x """
    return a1x  # Minimize braking force

if __name__ == "__main__":
    # Define initial conditions
    P1 = np.array([0, 0])  # Initial position of Object 1
    V1 = np.array([5, 2])  # Initial velocity of Object 1
    P2 = np.array([10, 10])  # Initial position of Object 2
    V2 = np.array([2, -1])  # Velocity of Object 2
    r1, r2 = 1, 1  # Radii of the objects
    t_final = 5  # Maximum time to check for collisions

    # Check if collision is even possible before optimization
    if detect_collision(P1, P2, V1, V2, r1, r2):
        print("Collision detected. Computing optimal braking force...")

        # Initial guess for deceleration
        a1x0 = [0.1]  # Small initial deceleration

        # Bounds: a1x in [0, max] to ensure non-negative braking
        bounds = [(0.0, 10000000)]  # Prevents infinite braking

        # Solve optimization problem
        res = minimize(
            objective,
            a1x0,
            bounds=bounds,
            constraints={'type': 'ineq', 'fun': constraint_no_collision, 'args': (P1, P2, V1, V2, r1, r2, t_final)},
            method="SLSQP"
        )

        # Optimal deceleration
        if res.success:
            optimal_a1x = res.x[0]
            print(f"Optimal Deceleration in x-direction: {optimal_a1x:.4f}")
        else:
            print("No feasible solution found. Consider adjusting parameters.")
    else:
        print("No collision detected. No braking required.")


Collision detected. Computing optimal braking force...
a1x [0.1]
a1x [0.1]
a1x [0.1]
a1x [0.10000001]
a1x [0.14386871]
a1x [0.10438687]
a1x [0.10438687]
a1x [0.10438689]
a1x [0.14369264]
a1x [0.10831745]
a1x [0.10831745]
a1x [0.10831746]
a1x [0.14355521]
a1x [0.11184122]
a1x [0.11184122]
a1x [0.11184124]
a1x [0.1434475]
a1x [0.11500185]
a1x [0.11500185]
a1x [0.11500187]
a1x [0.14336277]
a1x [0.11783794]
a1x [0.11783794]
a1x [0.11783796]
a1x [0.14329589]
a1x [0.12038374]
a1x [0.12038374]
a1x [0.12038375]
a1x [0.14324297]
a1x [0.12266966]
a1x [0.12266966]
a1x [0.12266968]
a1x [0.14320024]
a1x [0.12472272]
a1x [0.12472272]
a1x [0.12472273]
a1x [0.14318682]
a1x [0.12656913]
a1x [0.12656913]
a1x [0.12656914]
a1x [0.1431409]
a1x [0.12822631]
a1x [0.12822631]
a1x [0.12822632]
a1x [0.14311966]
a1x [0.12971564]
a1x [0.12971564]
a1x [0.12971566]
a1x [0.14310267]
a1x [0.13105435]
a1x [0.13105435]
a1x [0.13105436]
a1x [0.14308908]
a1x [0.13225782]
a1x [0.13225782]
a1x [0.13225783]
a1x [0.14307818]

In [35]:
import numpy as np
from scipy.optimize import minimize

def distance(P1, P2):
    """ Compute Euclidean distance between two points """
    return np.linalg.norm(P1 - P2)

def detect_collision(P1, P2, V1, V2, r1, r2):
    """
    Checks if a collision is possible based on initial relative motion.
    """
    delta_p = P2 - P1  # Displacement vector
    delta_v = V2 - V1  # Relative velocity

    # Check if objects are moving toward each other
    return np.dot(delta_p, delta_v) < 0

def estimate_collision_time(P1, P2, V1, V2, r1, r2):
    """
    Estimates the time of closest approach (if collision is possible).
    """
    delta_p = P2 - P1
    delta_v = V2 - V1

    if np.linalg.norm(delta_v) < 1e-6:
        return np.inf  # If objects have no relative velocity, they won't collide

    t_c = -np.dot(delta_p, delta_v) / np.linalg.norm(delta_v)**2

    if t_c < 0:
        return np.inf  # If t_c is negative, the objects were closer in the past

    return t_c

def constraint_no_collision(a1x, P1, P2, V1, V2, r1, r2):
    """
    Ensures Object 1 does not collide with Object 2 during movement.
    """

    # Estimate time of closest approach
    # t_c = estimate_collision_time(P1, P2, V1, V2, r1, r2)
    # if t_c == np.inf:
    #     return 1  # No collision risk

    # Simulate motion for multiple time steps up to t_c
    time_steps = np.linspace(0, min(500, 5), num=10000)  # Consider max 5 seconds

    for t in time_steps:
        # Update Object 1's position (only decelerating in x-direction)
        new_x = P1[0] + V1[0] * t - 0.5 * a1x[0] * t**2  # Apply braking in x direction
        new_y = P1[1] + V1[1] * t  # Y-direction remains unchanged
        P1_new = np.array([new_x, new_y])

        # Update Object 2's position (continues moving)
        P2_new = P2 + V2 * t

        # Check if collision occurs
        d = distance(P1_new, P2_new) - (r1 + r2)

        if d < 0:  # Collision detected
            return d  # Negative value signals violation

    return 1  # Constraint satisfied

def objective(a1x):
    """ Objective function: minimize deceleration a1x """
    return a1x  # Minimize braking force

if __name__ == "__main__":
    # Define initial conditions
    P1 = np.array([0, 0])  # Initial position of Object 1
    V1 = np.array([500, 500])  # Initial velocity of Object 1
    P2 = np.array([10, 10])  # Initial position of Object 2
    V2 = np.array([1, 1])  # Velocity of Object 2
    r1, r2 = 1, 1  # Radii of the objects

    # Check if a collision is even possible before optimization
    if detect_collision(P1, P2, V1, V2, r1, r2):
        print("Collision detected. Computing optimal braking force...")

        # Estimate time to collision
        t_final = estimate_collision_time(P1, P2, V1, V2, r1, r2)

        if t_final == np.inf:
            print("No risk of collision. No braking required.")
        else:
            print(f"Estimated time to collision: {t_final:.2f} seconds")

            # Initial guess for deceleration
            a1x0 = [0.1]  # Small initial deceleration

            # Bounds: a1x in [0, 50] to ensure non-negative braking
            bounds = [(0.0, 50)]  # Allow strong braking if needed

            # Solve optimization problem
            res = minimize(
                objective,
                a1x0,
                bounds=bounds,
                constraints={'type': 'ineq', 'fun': constraint_no_collision, 'args': (P1, P2, V1, V2, r1, r2)},
                method="SLSQP"
            )

            # Optimal deceleration
            if res.success:
                optimal_a1x = res.x[0]
                print(f"Optimal Deceleration in x-direction: {optimal_a1x:.4f}")
            else:
                print("No feasible solution found. Consider adjusting parameters.")
    else:
        print("No collision detected. No braking required.")


Collision detected. Computing optimal braking force...
Estimated time to collision: 0.02 seconds
No feasible solution found. Consider adjusting parameters.
